In [1]:
# ! conda install -c conda-forge imbalanced-learn
# ! pip install nose
# ! pip install imbalanced-ensemble           

In [22]:
import re

def read_dot_dat_file(path):
    datContent = [i.strip().split() for i in open(path).readlines()]
    r = re.compile("@inputs.*")
    _at_data = datContent.index(['@data'])
    assert datContent[0][0] == '@relation'
    assert datContent[_at_data-1][0] == '@outputs'
    assert datContent[_at_data-2][0] == '@inputs'
    print(datContent[_at_data-3][2:])
    assert len(datContent[_at_data-3][2:]) == 2   # Two Class

    col_names = datContent[_at_data-2][1:]
    col_names.append(datContent[_at_data-1][1])
    
    df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=r', ', engine='python')
    # df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=", ", engine='python')

    class1 = datContent[_at_data-3][2:][0].replace("{","").replace(",","")
    class2 = datContent[_at_data-3][2:][1].replace("}","").replace(",","")

    df['Class'] = df['Class'].replace({class1: 1, class2: -1})
    
    return df



# temp

In [21]:
# import pandas as pd 
# import numpy as np
# import os

# base_path = 'Datasets/'
# need_to_convert = ['ecoli-0_vs_1-5-fold','haberman-5-fold','page-blocks0-5-fold','pima-5-fold','vehicle2-5-fold']

# for needed in need_to_convert:
#     for datFile in os.listdir(base_path+needed):
#         print(base_path+needed+'/'+datFile)
#         df = read_dot_dat_file(base_path+needed+'/'+datFile)
#         df.to_excel(base_path+needed+'/'+datFile.split('.')[0]+'.xlsx')


In [3]:
import time
import os
import glob
import numpy as np
import random  
import pandas as pd 
import random

from sklearn.tree import DecisionTreeClassifier 

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc,f1_score,matthews_corrcoef
from imblearn.metrics import geometric_mean_score


from imblearn.ensemble import RUSBoostClassifier
from imbalanced_ensemble.ensemble import SMOTEBoostClassifier


In [7]:
base_path = 'Datasets/'
datasets  = os.listdir(base_path) 
classifiers = {"RUS": RUSBoostClassifier(random_state=0, algorithm='SAMME', estimator=DecisionTreeClassifier(max_depth=10)),
              "SMOTE": SMOTEBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10), n_estimators = 100, algorithm='SAMME', random_state=0)}


def clasify(dataset, classifier):
    mcc = []
    f1 = []
    auc_a = []
    gmean = []
    times = []
    traFiles = sorted(glob.glob(base_path+dataset+'/*tra.xlsx'))
    tstFiles = sorted(glob.glob(base_path+dataset+'/*tst.xlsx'))
    print(traFiles)
    for traPath, tstPath in zip(traFiles, tstFiles):
        
        df_train = pd.read_excel(traPath)
        df_test = pd.read_excel(tstPath)

        x_train= df_train.iloc[:, 0:-1]
        y_train = df_train.iloc[:, -1]
        x_test= df_test.iloc[:, 0:-1]
        y_test = df_test.iloc[:, -1]
        
        st = time.time()
        clf = classifiers[classifier]

        clf.fit(x_train, y_train)

        y_pred = clf.predict(x_test) 
        et = time.time()
        
        # compute error
        mcc.append(matthews_corrcoef(y_test, y_pred))
        #--------------------------------
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auc_a.append(auc(fpr, tpr))
        #--------------------------------
        f1.append(f1_score(y_test, y_pred))
        #--------------------------------
        gmean.append(geometric_mean_score(y_test, y_pred, labels=[1, -1]))
        
        #time of train and test
        times.append(et - st)
        
    return {"mcc": mcc, "auc": auc_a, "f1": f1, "gmean": gmean, "exe_time": times}

clasify(datasets[0], 'SMOTE')

['Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-1tra.xlsx', 'Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-2tra.xlsx', 'Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-3tra.xlsx', 'Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-4tra.xlsx', 'Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-5tra.xlsx']


{'mcc': [0.19062941194383556,
  0.028416430811269728,
  0.10852713178294573,
  0.4453928491860374,
  0.15819888872077387],
 'auc': [0.6201550387596899,
  0.5193798449612403,
  0.554263565891473,
  0.7926356589147286,
  0.6078431372549019],
 'f1': [0.25,
  0.1111111111111111,
  0.16666666666666666,
  0.4705882352941177,
  0.2222222222222222],
 'gmean': [0.5498414147691576,
  0.3812464258315117,
  0.3962029078465307,
  0.7825618830323859,
  0.5423261445466404],
 'exe_time': [1.0054960250854492,
  0.9937365055084229,
  1.2587642669677734,
  0.8357181549072266,
  0.854461669921875]}